In [11]:

import os
import sys

project_root = "/Users/thechuongchu/Desktop/SCOPIC/Tornet_services/"
sys.path.append(project_root)


In [30]:

from src.model import TornadoLikelihood
import torch
model = TornadoLikelihood(
            shape=(2, 720, 1152),
            c_shape=(2, 720, 1152),
            input_variables=['DBZ',
                'VEL', 
                'KDP',
                'RHOHV',
                'ZDR',
                'WIDTH'],
            start_filters=48,
            background_flag=-3.0,
            include_range_folded=True
    )
model_path = "../model_checkpoint/tornet.pth"
state_dict = torch.load(model_path)
model.load_state_dict(state_dict)
model.eval()

/var/folders/h7/f7qx4yhs1qg76t3j70vf441c0000gn/T/ipykernel_71973/103339836.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)


TornadoLikelihood(
  (blk1): VggBlock(
    (conv_block): Sequential(
      (0): CoordConv2D(
        (conv): Conv2d(16, 48, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (conv_activation): ReLU()
      )
      (1): CoordConv2D(
        (conv): Conv2d(50, 48, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (conv_activation): ReLU()
      )
    )
    (mx): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (mc): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (drop): Dropout(p=0.1, inplace=False)
  )
  (blk2): VggBlock(
    (conv_block): Sequential(
      (0): CoordConv2D(
        (conv): Conv2d(50, 96, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (conv_activation): ReLU()
      )
      (1): CoordConv2D(
        (conv): Conv2d(98, 96, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (conv_activation): ReLU()
      )
    )
    (mx): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, c

In [37]:
import numpy as np

# Example batch size
batch_size = 1
az, rng = 720, 1152  # Matching input shape

# Create dummy radar data with the correct shape
dummy_data = {
    v: torch.randn(batch_size, 2, az, rng) for v in model.input_variables
}
dummy_data["coordinates"] = torch.randn(batch_size, 2, az, rng)

# Add range_folded_mask if used
if model.include_range_folded:
    dummy_data["range_folded_mask"] = torch.randn(batch_size, 2, az, rng)

In [32]:
class ONNXWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, *args):
        # Convert list of args back to dictionary
        keys = self.model.input_variables + ["coordinates"]
        if self.model.include_range_folded:
            keys.append("range_folded_mask")

        data_dict = {k: v for k, v in zip(keys, args)}
        return self.model(data_dict)


In [33]:
onnx_model = ONNXWrapper(model)


In [20]:
dummy_input_tuple = tuple(dummy_data[k] for k in dummy_data.keys())

# Export to ONNX
torch.onnx.export(
    onnx_model, 
    dummy_input_tuple, 
    "tornado_likelihood.onnx", 
    export_params=True,  # Store trained parameters
    opset_version=12,    # ONNX opset version
    do_constant_folding=True,  # Optimize constants
    input_names=list(dummy_data.keys()), 
    output_names=['likelihood_map']
)

print("Model successfully converted to ONNX!")


Model successfully converted to ONNX!


In [39]:
import onnx

onnx_model = onnx.load("tornado_likelihood.onnx")
onnx.checker.check_model(onnx_model)


In [47]:
dummy_data['VEL'].shape

torch.Size([1, 2, 720, 1152])

In [43]:
import onnxruntime
import numpy as np

# Create random input data with correct shapes
random_input = {}
for k, v in dummy_data.items():
    random_input[k] = np.random.randn(*v.shape).astype(np.float32)

# Run inference with PyTorch model
torch_inputs = {k: torch.from_numpy(v) for k, v in random_input.items()}
with torch.no_grad():
    torch_output = model(torch_inputs)  # Ensure inputs are tensors for the PyTorch model
    
# Run inference with ONNX model
ort_session = onnxruntime.InferenceSession("tornado_likelihood.onnx")
ort_inputs = {k.name: random_input[k.name] for k in ort_session.get_inputs()}
ort_output = ort_session.run(None, ort_inputs)[0]

print("PyTorch output shape:", torch_output.shape)
print("ONNX output shape:", ort_output.shape)

# Compare outputs
torch_output_np = torch_output.numpy() if isinstance(torch_output, torch.Tensor) else torch_output
print("\nMax absolute difference:", np.abs(torch_output_np - ort_output).max())
print("Mean absolute difference:", np.abs(torch_output_np - ort_output).mean())


PyTorch output shape: torch.Size([1, 1, 45, 72])
ONNX output shape: (1, 1, 45, 72)

Max absolute difference: 1.9073486e-05
Mean absolute difference: 2.3238454e-06
